In [ ]:
!pip install --upgrade scipy tensorflow tensorflow-federated==0.41 pandas scikit-learn cryptography google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client

  Using cached scipy-1.14.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (60 kB)
  Using cached tensorflow-2.17.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.2 kB)
  Using cached tensorflow_federated-0.41.0-py2.py3-none-any.whl.metadata (3.8 kB)
  Using cached google_auth_oauthlib-1.2.1-py2.py3-none-any.whl.metadata (2.7 kB)
  Using cached attrs-21.4.0-py2.py3-none-any.whl.metadata (9.8 kB)
  Using cached cachetools-3.1.1-py2.py3-none-any.whl.metadata (4.6 kB)
  Using cached dm_tree-0.1.7-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (1.9 kB)
  Using cached dp_accounting-0.3.0-py3-none-any.whl.metadata (1.7 kB)
  Using cached farmhashpy-0.4.0.tar.gz (98 kB)
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of tensorflow-federated to determine which version is compatible with other requirements. This could take a while.
ERROR: Ignored the following yanked versions: 0.4.32
ERROR: Igno

In [ ]:
!pip install tenseal


In [54]:
import tensorflow as tf
import tensorflow_federated as tff
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os
import numpy as np
from google.colab import auth
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from google.auth.transport.requests import Request
import google.auth
import base64
import tenseal as ts

# Authenticate and create the Google Drive client
print("Authenticating and creating Google Drive client...")
print("Client : Anwarul")
auth.authenticate_user()
creds, _ = google.auth.default()
if creds.expired and creds.refresh_token:
    creds.refresh(Request())
print("Authentication successful. Connected to Google Drive.")

drive_service = build('drive', 'v3', credentials=creds)
print("Google Drive is set as the global server.")

# Load the dataset directly from the uploaded file
data_path = '/content/johns_hospital_data.csv'  # Change filename as needed
print(f"Loading dataset from {data_path}...")
data = pd.read_csv(data_path)

# Data Preprocessing
print("Starting data preprocessing...")
data_cleaned = data.drop(['Admission_date', 'hospital_name', 'patient_id',
                         'patient_first_initial', 'patient_last_name',
                         'doctor_name', 'patient_checkin_date', 'patient_checkout_date'], axis=1)

# Convert categorical columns to numerical using Label Encoding
label_encoder = LabelEncoder()
categorical_columns = data_cleaned.select_dtypes(include=['object']).columns

for column in categorical_columns:
    data_cleaned[column] = label_encoder.fit_transform(data_cleaned[column])

# Define features and target variable
X = data_cleaned.drop('readmission', axis=1)
y = data_cleaned['readmission']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Data split into training and testing sets.")

# Standardize the dataset
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to TensorFlow Dataset
train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32)

# Prepare the dataset for federated learning
def preprocess(dataset):
    return dataset.map(lambda x, y: (tf.cast(x, tf.float32), tf.cast(y, tf.int32))) \
                  .map(lambda x, y: (x, tf.expand_dims(y, -1)))

train_dataset = preprocess(train_dataset)
print("Data preprocessing completed successfully.")

# Define the model function for Federated Learning
def create_keras_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),  # Ensure input shape is correct
        tf.keras.layers.Dense(1, activation='sigmoid')  # Binary classification output
    ])

def model_fn():
    keras_model = create_keras_model()
    return tff.learning.models.from_keras_model(
        keras_model,
        input_spec=(tf.TensorSpec(shape=[None, X_train.shape[1]], dtype=tf.float32),
                    tf.TensorSpec(shape=[None, 1], dtype=tf.int32)),
        loss=tf.keras.losses.BinaryCrossentropy(),
        metrics=[tf.keras.metrics.BinaryAccuracy()])

print("Model function defined successfully.")

# Homomorphic Encryption Setup
def encrypt_weights_homomorphic(weights):
    encrypted_weights = []

    print("Encrypting model weights using homomorphic encryption...")
    context = ts.context(ts.SCHEME_TYPE.CKKS, poly_modulus_degree=8192, coeff_mod_bit_sizes=[60, 40, 40, 60])
    context.generate_galois_keys()
    context.global_scale = 2**40

    for w in weights:
        data = w.flatten().tolist()
        encrypted_data = ts.ckks_vector(context, data)
        encrypted_weights.append(encrypted_data)

    print("Model weights encrypted successfully using homomorphic encryption.")
    return encrypted_weights

def decrypt_weights_homomorphic(encrypted_weights):
    decrypted_weights = []

    print("Decrypting model weights using homomorphic encryption...")
    for encrypted_data in encrypted_weights:
        decrypted_data = encrypted_data.decrypt()
        decrypted_weights.append(np.array(decrypted_data))

    print("Model weights decrypted successfully using homomorphic encryption.")
    return decrypted_weights

# Example of encrypting model weights using homomorphic encryption
model_weights = create_keras_model().get_weights()
encrypted_weights_homomorphic = encrypt_weights_homomorphic([w.flatten() for w in model_weights])

# Decrypt weights when needed
decrypted_weights_homomorphic = decrypt_weights_homomorphic(encrypted_weights_homomorphic)

# Reshape the decrypted weights to match the original shapes
decrypted_weights_homomorphic = [w.reshape(original_w.shape) for w, original_w in zip(decrypted_weights_homomorphic, model_weights)]
print("Model weights encryption and decryption completed successfully using homomorphic encryption.")

# Proceed with federated learning setup
try:
    print("Building federated averaging process...")
    client_optimizer_fn = lambda: tf.keras.optimizers.SGD(learning_rate=0.01)

    iterative_process = tff.learning.algorithms.build_weighted_fed_avg(
        model_fn=model_fn,
        client_optimizer_fn=client_optimizer_fn
    )
    state = iterative_process.initialize()
    print("Federated averaging process initialized successfully.")

    # Indicating connection between clients and global server
    print("Clients are connected to the global server for federated learning.")

    # Perform multiple rounds of federated learning
    for round_num in range(1, 11):
        print(f"Starting round {round_num} of federated learning...")
        state, metrics = iterative_process.next(state, [train_dataset])
        print(f'Round {round_num} metrics: {metrics}')

    print("Federated learning rounds completed successfully.")
except AttributeError as ae:
    print("AttributeError:", ae)
except Exception as e:
    print("An error occurred:", e)

# Save Model Updates to Google Drive in the new Keras format
save_path = '/tmp/Anwarul_model_updates.keras'
model_to_save = create_keras_model()  # Create the model
model_to_save.set_weights(decrypted_weights_homomorphic)  # Set weights if using encryption
model_to_save.save(save_path)
print(f"Model saved locally at {save_path}.")

file_metadata = {
    'name': 'Anwarul_model_updates.keras',
    'parents': ['1WVQMK27bObX0N0hDntjz1peL8bTHvsi7']  # Your global server folder ID
}
media = MediaFileUpload(save_path, mimetype='application/octet-stream')
uploaded_file = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
print(f"Model saved to Google Drive with ID: {uploaded_file.get('id')}.")

# Load Model Updates
loaded_model = tf.keras.models.load_model(save_path)
loaded_weights = loaded_model.get_weights()
print("Loaded model weights from Google Drive.")

# Comparing Loaded and Decrypted Weights
def compare_weights(loaded_weights, decrypted_weights):
    if len(loaded_weights) != len(decrypted_weights):
        return False
    for lw, dw in zip(loaded_weights, decrypted_weights):
        if lw.shape != dw.shape or not np.allclose(lw, dw, rtol=1e-5, atol=1e-8):
            return False
    return True

if compare_weights(loaded_weights, decrypted_weights_homomorphic):
    print("Weights loaded correctly and match the saved weights.")
else:
    print("Discrepancy found in loaded weights.")

# List files in the global server folder
results = drive_service.files().list(
    q=f"'1WVQMK27bObX0N0hDntjz1peL8bTHvsi7' in parents",
    pageSize=10, fields="files(id, name)").execute()
items = results.get('files', [])

if not items:
    print("No files found in the global server folder.")
else:
    print("Files found in the global server folder:")
    for item in items:
        print(f"{item['name']} ({item['id']})")

def privacy_measure(original_weights, encrypted_weights):
    original_sum = sum(np.sum(w) for w in original_weights)
    decrypted_weights = decrypt_weights_homomorphic(encrypted_weights)
    decrypted_sum = sum(np.sum(w) for w in decrypted_weights)

    if np.isclose(original_sum, decrypted_sum):
        print("Privacy is preserved!!")
    else:
        print("Privacy is compromised!!")

privacy_measure(model_weights, encrypted_weights_homomorphic)

Authenticating and creating Google Drive client...
Client : Anwarul
Authentication successful. Connected to Google Drive.
Google Drive is set as the global server.
Loading dataset from /content/johns_hospital_data.csv...
Starting data preprocessing...
Data split into training and testing sets.
Data preprocessing completed successfully.
Model function defined successfully.
Encrypting model weights using homomorphic encryption...
Model weights encrypted successfully using homomorphic encryption.
Decrypting model weights using homomorphic encryption...
Model weights decrypted successfully using homomorphic encryption.
Model weights encryption and decryption completed successfully using homomorphic encryption.
Building federated averaging process...
Federated averaging process initialized successfully.
Clients are connected to the global server for federated learning.
Starting round 1 of federated learning...
Round 1 metrics: OrderedDict([('distributor', ()), ('client_work', OrderedDict([(